In [39]:
# Note on all the variables used in this code
# PCT_ACS19_5yr_B08134081,% Total:  Subway Or Elevated Rail Light Rail Streetcar Or Trolley


In [4]:
# import packages
import pandas as pd
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [5]:
# import helper functions
from helpers.variable_helper import derive_variables

In [6]:
# Retrieve data and derive appropriate variables
df = pd.read_csv('./data/2019_block_group_data/raw_data/ny_2019_acs_block_group_data_zeroed.csv', na_values=".", skiprows=[0])

In [7]:
derive_variables(df)

In [8]:
df

,Geo_FIPS,Geo_AREALAND,Geo_AREAWATR,ACS19_5yr_B01001001,ACS19_5yr_B01001002,ACS19_5yr_B01001003,ACS19_5yr_B01001004,ACS19_5yr_B01001005,ACS19_5yr_B01001006,ACS19_5yr_B01001007,...,PCT_ACS19_5yr_B23025003,PCT_ACS19_5yr_B23025004,PCT_ACS19_5yr_B23025005,PCT_ACS19_5yr_B23025006,PCT_ACS19_5yr_B23025007,ACS19_5yr_B19013001,ACS19_5yr_B19013A001,ACS19_5yr_B19013B001,ACS19_5yr_B19013D001,population_density
0,360010001001,2010546.0,245155.0,760,364,43,5,16,11,0,...,63.85,51.96,11.89,0.47,35.68,44821.0,0,0,0,0.000378
1,360010001002,356866.0,0.0,1275,547,33,35,21,107,15,...,62.13,59.57,2.56,0.00,37.87,30361.0,0,0,0,0.003573
2,360010002001,1449342.0,0.0,2188,994,155,93,127,0,0,...,69.46,59.03,10.43,0.00,30.54,24271.0,0,0,0,0.001510
3,360010002002,206624.0,0.0,508,86,0,0,0,0,0,...,95.90,95.90,0.00,0.00,4.10,43220.0,0,0,0,0.002459
4,360010002003,216940.0,0.0,1350,736,13,40,0,96,0,...,51.39,43.04,8.35,0.00,48.61,24646.0,0,0,0,0.006223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15458,361231504004,734110.0,48061.0,569,338,0,18,0,9,7,...,53.77,53.77,0.00,0.00,46.23,60625.0,0,0,0,0.000775
15459,361231505001,81198648.0,43784.0,1650,805,56,62,75,49,25,...,60.42,57.75,2.67,0.00,39.58,65054.0,0,0,0,0.000020
15460,361231505002,41094436.0,8395189.0,883,446,23,3,9,10,2,...,40.76,40.13,0.64,0.00,59.24,72679.0,0,0,0,0.000021
15461,361231505003,24333240.0,15535.0,1139,420,10,18,25,11,0,...,46.64,44.54,2.10,0.00,53.36,36250.0,0,0,0,0.000047


In [ ]:
# remove Geo_FIPS as it's not needed in Lasso Regression
df = df.iloc[: , 1:]

In [ ]:
from sklearn.model_selection import train_test_split
scaler = preprocessing.StandardScaler()
y = df['PCT_ACS19_5yr_B08134081']
X = df.loc[:, df.columns != 'PCT_ACS19_5yr_B08134081']
# X = df.drop(['pct_walking_and_public_transit'], axis=1, inplace=True)

In [ ]:
import numpy as np
df.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
df=df.dropna()
df=df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
# df = df.replace(np.nan, 0)
# df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
df.shape

(15463, 621)

In [ ]:
df

,Geo_AREALAND,Geo_AREAWATR,ACS19_5yr_B01001001,ACS19_5yr_B01001002,ACS19_5yr_B01001003,ACS19_5yr_B01001004,ACS19_5yr_B01001005,ACS19_5yr_B01001006,ACS19_5yr_B01001007,ACS19_5yr_B01001008,...,PCT_ACS19_5yr_B23025002,PCT_ACS19_5yr_B23025003,PCT_ACS19_5yr_B23025004,PCT_ACS19_5yr_B23025005,PCT_ACS19_5yr_B23025006,PCT_ACS19_5yr_B23025007,ACS19_5yr_B19013001,ACS19_5yr_B19013A001,ACS19_5yr_B19013B001,ACS19_5yr_B19013D001
0,2010546.0,245155.0,760,364,43,5,16,11,0,0,...,64.32,63.85,51.96,11.89,0.47,35.68,44821.0,0,0,0
1,356866.0,0.0,1275,547,33,35,21,107,15,31,...,62.13,62.13,59.57,2.56,0.00,37.87,30361.0,0,0,0
2,1449342.0,0.0,2188,994,155,93,127,0,0,0,...,69.46,69.46,59.03,10.43,0.00,30.54,24271.0,0,0,0
3,206624.0,0.0,508,86,0,0,0,0,0,0,...,95.90,95.90,95.90,0.00,0.00,4.10,43220.0,0,0,0
4,216940.0,0.0,1350,736,13,40,0,96,0,152,...,51.39,51.39,43.04,8.35,0.00,48.61,24646.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15458,734110.0,48061.0,569,338,0,18,0,9,7,0,...,53.77,53.77,53.77,0.00,0.00,46.23,60625.0,0,0,0
15459,81198648.0,43784.0,1650,805,56,62,75,49,25,15,...,60.42,60.42,57.75,2.67,0.00,39.58,65054.0,0,0,0
15460,41094436.0,8395189.0,883,446,23,3,9,10,2,11,...,40.76,40.76,40.13,0.64,0.00,59.24,72679.0,0,0,0
15461,24333240.0,15535.0,1139,420,10,18,25,11,0,27,...,46.64,46.64,44.54,2.10,0.00,53.36,36250.0,0,0,0


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

# https://towardsdatascience.com/feature-selection-in-machine-learning-using-lasso-regression-7809c7c2771a


In [ ]:
# pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )
search.fit(X_train,y_train)


Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END .................model__alpha=0.1;, score=-0.848 total time=   2.0s
[CV 2/5] END .................model__alpha=0.1;, score=-0.442 total time=   2.1s
[CV 3/5] END .................model__alpha=0.1;, score=-0.760 total time=   2.0s
[CV 4/5] END .................model__alpha=0.1;, score=-0.682 total time=   2.0s
[CV 5/5] END .................model__alpha=0.1;, score=-0.859 total time=   1.9s
[CV 1/5] END .................model__alpha=0.2;, score=-2.632 total time=   1.1s
[CV 2/5] END .................model__alpha=0.2;, score=-1.609 total time=   1.7s
[CV 3/5] END .................model__alpha=0.2;, score=-2.387 total time=   1.3s
[CV 4/5] END .................model__alpha=0.2;, score=-2.229 total time=   1.2s
[CV 5/5] END .................model__alpha=0.2;, score=-2.555 total time=   1.1s
[CV 1/5] END .model__alpha=0.30000000000000004;, score=-2.921 total time=   0.7s
[CV 2/5] END .model__alpha=0.30000000000000004;

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [ ]:
search.best_params_

{'model__alpha': 0.1}

In [ ]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [ ]:
importance = np.abs(coefficients)

In [ ]:
result = np.array(df.loc[:, df.columns != 'PCT_ACS19_5yr_B08134081'].columns)[importance > 0]

In [ ]:
df_columns = pd.read_csv('./raw_data/2019_data_columns.csv', na_values=".")
df_columns

,name,decode
0,Geo_FIPS,FIPS
1,Geo_NAME,Area Name
2,Geo_QName,Qualifying Name
3,Geo_STUSAB,State Postal Abbreviation
4,Geo_SUMLEV,Summary Level
...,...,...
988,ACS19_5yr_B19013A001s,Std. Error: Median Household Income In The Pas...
989,ACS19_5yr_B19013B001,Median Household Income In The Past 12 Months ...
990,ACS19_5yr_B19013B001s,Std. Error: Median Household Income In The Pas...
991,ACS19_5yr_B19013D001,Median Household Income In The Past 12 Months ...


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
df_columns.loc[df_columns['name'].isin(result)]

,name,decode
460,PCT_ACS19_5yr_B08134011,% Total: Car Truck Or Van
510,PCT_ACS19_5yr_B08134061,% Total: Public Transportation (Excluding Taxicab)
531,PCT_ACS19_5yr_B08134082,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): Less Than 10 Minutes
532,PCT_ACS19_5yr_B08134083,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 10 To 14 Minutes
533,PCT_ACS19_5yr_B08134084,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 15 To 19 Minutes
534,PCT_ACS19_5yr_B08134085,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 20 To 24 Minutes
535,PCT_ACS19_5yr_B08134086,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 25 To 29 Minutes
536,PCT_ACS19_5yr_B08134087,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 30 To 34 Minutes
537,PCT_ACS19_5yr_B08134088,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 35 To 44 Minutes
538,PCT_ACS19_5yr_B08134089,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 45 To 59 Minutes


In [ ]:
df_columns.loc[df_columns['name'].isin(result)].iloc[:,0].values.tolist()

['PCT_ACS19_5yr_B08134011',
 'PCT_ACS19_5yr_B08134061',
 'PCT_ACS19_5yr_B08134082',
 'PCT_ACS19_5yr_B08134083',
 'PCT_ACS19_5yr_B08134084',
 'PCT_ACS19_5yr_B08134085',
 'PCT_ACS19_5yr_B08134086',
 'PCT_ACS19_5yr_B08134087',
 'PCT_ACS19_5yr_B08134088',
 'PCT_ACS19_5yr_B08134089',
 'PCT_ACS19_5yr_B08134090',
 'PCT_ACS19_5yr_B08134091',
 'ACS19_5yr_B08301021',
 'PCT_ACS19_5yr_B08301012',
 'PCT_ACS19_5yr_B08301014',
 'PCT_ACS19_5yr_B08301021']